In [1]:
! pip install jupyter-dash


   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------- ----------------------------- 2.1/7.8 MB 11.7 MB/s eta 0:00:01
   ---------------- ----------------------- 3.1/7.8 MB 10.2 MB/s eta 0:00:01
   ------------------------------ --------- 6.0/7.8 MB 10.0 MB/s eta 0:00:01
   ---------------------------------------  7.6/7.8 MB 9.8 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 8.3 MB/s eta 0:00:00


In [3]:
! pip install dash-uploader


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for dash-uploader: filename=dash_uploader-0.6.1-py3-none-any.whl size=70339 sha256=5f2077dc833abd1a1182dd054d70a2c43a269ed19b5eb7c2cd5bc62aa5f2d67a
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\5f\c3\21\61f26ad92b73b2e53af6309e7b20be86bf7f8845ff6338596c
Successfully built dash-uploader


In [5]:
! pip install opencv-python


  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   - -------------------------------------- 1.8/39.5 MB 11.2 MB/s eta 0:00:04
   --- ------------------------------------ 3.1/39.5 MB 11.5 MB/s eta 0:00:04
   ----- ---------------------------------- 5.8/39.5 MB 9.3 MB/s eta 0:00:04
   ------- -------------------------------- 7.1/39.5 MB 8.6 MB/s eta 0:00:04
   -------- ------------------------------- 8.7/39.5 MB 8.5 MB/s eta 0:00:04
   ---------- ----------------------------- 10.2/39.5 MB 8.2 MB/s eta 0:00:04
   ----------- ---------------------------- 11.8/39.5 MB 8.0 MB/s eta 0:00:04
   ------------- -------------------------- 13.1/39.5 MB 7.8 MB/s eta 0:00:04
   -------------- ------------------------- 14.7/39.5 MB 7.8 MB/s eta 0:00:04
   ---------------- ----------------------- 16.5/39.5 MB 7.8 MB/s eta 0:00:03
   ------------------ --------------------- 18.1/39.5 MB 7.8 MB/s eta 0:0

In [7]:
! pip install tensorflow


In [1]:
import dash
from dash import dcc, html, Input, Output, State
import dash_uploader as du
import os
import tensorflow as tf
import numpy as np
import cv2
import base64
from io import BytesIO
from PIL import Image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

# Load pre-trained VGG16 model
model = VGG16(weights='imagenet')

# Initialize Dash app
app = dash.Dash(__name__)
du.configure_upload(app, "./uploads")

app.layout = html.Div([
    html.H1("VGG16 Image Classification", style={"textAlign": "center", "color": "#4A90E2", "marginBottom": "20px"}),
    
    html.Div([
        dcc.Upload(
            id='upload-image',
            children=html.Div([
                html.Button('Upload Image', className="upload-btn", style={
                    "backgroundColor": "#FF5733", "color": "white", "border": "none", "padding": "12px 24px", "fontSize": "18px", "cursor": "pointer", "borderRadius": "8px", "fontWeight": "bold"
                })
            ]),
            multiple=False,
            style={"textAlign": "center"}
        ),
        html.Br(),
        html.Div(
            [
                html.Img(id="uploaded-image", style={"maxWidth": "300px", "display": "none", "margin": "auto", "border": "2px solid #ddd", "borderRadius": "10px", "padding": "10px"})
            ],
            style={"textAlign": "center", "border": "1px solid #ccc", "borderRadius": "10px", "padding": "15px", "width": "50%", "margin": "auto"}
        ),
        html.Br(),
        html.Button("Predict", id="predict-button", n_clicks=0, className="predict-btn", style={
            "backgroundColor": "#33C3FF", "color": "white", "border": "none", "padding": "12px 24px", "fontSize": "18px", "cursor": "pointer", "borderRadius": "8px", "fontWeight": "bold"
        }),
    ], style={"textAlign": "center", "padding": "20px"}),
    
    html.Div(id="prediction-output", style={"textAlign": "center", "fontSize": "18px", "marginTop": "20px", "padding": "10px", "border": "1px solid #ddd", "borderRadius": "10px", "width": "50%", "margin": "auto", "backgroundColor": "#f9f9f9"})
])

@app.callback(
    [Output("uploaded-image", "src"), Output("uploaded-image", "style")],
    [Input("upload-image", "contents")]
)
def display_image(contents):
    if contents:
        return contents, {"maxWidth": "300px", "display": "block", "margin": "auto", "border": "2px solid #ddd", "borderRadius": "10px", "padding": "10px"}
    return None, {"display": "none"}

@app.callback(
    Output("prediction-output", "children"),
    [Input("predict-button", "n_clicks")],
    [State("upload-image", "contents")]
)
def predict(n_clicks, contents):
    if n_clicks > 0 and contents:
        try:
            # Convert base64 image to PIL Image
            encoded_image = contents.split(",")[1]
            decoded_image = base64.b64decode(encoded_image)
            image = Image.open(BytesIO(decoded_image))
            image = image.resize((224, 224))
            image = np.array(image)
            if image.shape[-1] == 4:
                image = image[:, :, :3]  # Convert RGBA to RGB
            image = np.expand_dims(image, axis=0)
            image = preprocess_input(image)
            
            # Predict
            preds = model.predict(image)
            decoded_preds = decode_predictions(preds, top=3)[0]
            results = [f"{i+1}. {label} ({prob:.2f})" for i, (_, label, prob) in enumerate(decoded_preds)]
            return html.Ul([html.Li(result, style={"fontSize": "20px", "color": "#333", "margin": "5px 0"}) for result in results])
        except Exception as e:
            return f"Error processing image: {str(e)}"
    return "Upload an image and click Predict."

if __name__ == "__main__":
    app.run_server(debug=True, port=8060)
